In [1]:
import os
import sys
import numpy as np
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time
from urllib import request
from urllib.request import urlretrieve
import librosa
from librosa import display
import matplotlib.pyplot as plt
import skimage
from shutil import copy2
import random
from pathlib import Path

In [5]:
track_filepath = os.path.join('data', "metal_subgenre_data.csv")
df = pd.read_csv(track_filepath)
del df['Unnamed: 0']
del df['Unnamed: 0.1']
del df['index']
df.drop_duplicates(subset="track_id", inplace=True)

EmptyDataError: No columns to parse from file

In [ ]:
df['genre'].unique()

In [ ]:
#get authorization to access spotify

def spotify_login(cid, secret):
    client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
    return spotipy.Spotify(client_credentials_manager=client_credentials_manager)

cid = "############################" 
secret = "############################" 

sp = spotify_login(cid, secret)

In [ ]:
#download track previews from urls in dataframe

mp3_folder = os.path.join(sys.path[0], "/metal_track_previews/")

preview_unavailable = []

i = 0
skip = 0
for index, row in df.iterrows():
    track_id = str(row['track_id'])
    track_url = row['preview_url'] 
    prefix = track_id[0:2] + "/"
    full_directory = mp3_folder + prefix
    
    if track_url:
        
        #check if the mp3 file already exists
        if not os.path.exists(full_directory + track_id + ".mp3"):
            
            #make the folder if we don't have it alreadt
            if not os.path.isdir(full_directory):
                os.makedirs(full_directory)
            
            #try to download the track
            try:
                urlretrieve(track_url, full_directory + track_id + ".mp3")
                i += 1
                if i % 1000 == 0:
                    print(str(i) + " tracks downloaded.")
            except Exception as e:
                print(e)
                print("Skipping track.")
                preview_unavailable.append(int(index))
              
    else:
        preview_unavailable.append(int(index))

num_downloaded = len(df.index) - len(preview_unavailable)
print("Downloaded " +  str(num_downloaded) + "/" + str(len(df.index)) + " mp3 previews.")

#df.drop(df.index[preview_unavailable], inplace=True)
#df.reset_index(drop=True, inplace=True)